## Import packages

In [1]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import matplotlib.pyplot as plt

import cv2
import numpy as np
import torch
import os.path
import glob
import pandas as pd
import random
from math import sqrt


from PIL import Image

from vrProjector_master import vrProjector
from skimage import morphology

from torchvision.transforms import transforms
from torchvision.utils import make_grid, save_image
from torch.utils.tensorboard import SummaryWriter

# Image processing functions

In [2]:
# turn original renderings to small size (128) grayscale images, not necessary for a single image 
def To_Gray_Label_given_list(png_list,dir_out):
    i=0
    for pngfile in png_list: #rendered PNG path
        
        #read images
        input_name = str(pngfile)[3:]
        example=cv2.imread(pngfile, cv2.IMREAD_COLOR)
        output_name = dir_out+input_name

        #k-means clustering
        img_data = example / 255.0
        img_data = img_data.reshape((-1, 3))

        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,100000, 0.00001)
        flags = cv2.KMEANS_RANDOM_CENTERS
        img_data = img_data.astype(np.float32)
        compactness, labels, centers = cv2.kmeans(img_data,5, None, criteria,10, flags)#number of clusters (classes: 5)

        centers[:,:] = np.round(centers[:,:])
        new_colors = centers[labels].reshape((-1, 3))

        example_recolored = new_colors.reshape(example.shape) #recolored sample

        #RGB map to grayscale map
        import pandas as pd
        w=example.shape[0]
        palette = np.array([[0, 0, 0],[0, 1, 1],[0, 0, 1],[0, 1, 0],[1, 1, 1]]) #order: black edge, ground, opaque building, glazing, sky
        matches = np.array([j for i in range(len(centers)) for j in range(len(palette)) if centers[i].tolist()==palette[j].tolist()]).reshape(-1,1)
        labels_ordered = matches[labels].reshape((w,w))
        labels_gray = np.array(labels_ordered*0.25)
        gray_img = cv2.cvtColor(labels_gray.astype(np.float32), cv2.COLOR_GRAY2BGR)#grayscale sample

        resized = cv2.resize(gray_img, (128,128), interpolation = cv2.INTER_NEAREST)#resze to 128*128
        
        print(output_name)
        
        cv2.imwrite(output_name,resized*256)

In [ ]:
# turn fisheye renderings to cube

In [3]:
def gen_pure_blk(w,h):

        b = np.zeros((w,h), dtype=np.uint8)
        g = np.zeros((w,h), dtype=np.uint8)
        r = np.zeros((w,h), dtype=np.uint8)

        blk = cv2.merge([b, g, r])
        
        return blk

In [4]:
def fisheye2dual(w,h,dir_r,dir_s,dir_o):
    
    for pngref in glob.glob(dir_r): #rendered PNG path
        
        #read images
        
        input_name = str(pngref)[11:]
        pngfile = dir_s + input_name
        
        example=cv2.imread(pngfile, cv2.IMREAD_COLOR)
        output_name = dir_o+input_name
       
        blk1 = gen_pure_blk(512,512)
        out=np.hstack([blk1, example])
        cv2.imwrite('test.PNG', out)

        source = vrProjector.SideBySideFisheyeProjection()
        source.loadImage("test.PNG")

        #source.set_use_bilinear(True)

        out = vrProjector.CubemapProjection()
        out.initImages(w,h)
        out.reprojectToThis(source)
        out.saveImages("cubemap/left.png", "cubemap/front.png", "cubemap/right.png", "cubemap/blk.png", "cubemap/top.png", "cubemap/bottom.png")

        #rotate_clockwise = cv2.getRotationMatrix2D((0,0), -90, 1.0)
        #rotate_counterclock = cv2.getRotationMatrix2D((0,0), 90, 1.0)

        front=cv2.imread("cubemap/front.png", cv2.IMREAD_COLOR)
        left=cv2.imread("cubemap/left.png", cv2.IMREAD_COLOR)
        right=cv2.imread("cubemap/right.png", cv2.IMREAD_COLOR)

        top2rotate=cv2.imread("cubemap/top.png", cv2.IMREAD_COLOR)
        #top = np.rot90(top2rotate,1) ##uncheck if there is bug
        top = np.rot90(top2rotate,-1) ##check if there is bug

        bottom2rotate=cv2.imread("cubemap/bottom.png", cv2.IMREAD_COLOR)
        bottom = np.rot90(bottom2rotate,1)

        blk2 = gen_pure_blk(w,h)

        col_left  = np.vstack([blk2, left, blk2])
        col_mid = np.vstack([top, front, bottom])
        col_right  = np.vstack([blk2, right, blk2])

        img = np.hstack([col_left, col_mid, col_right])

        img_cropped = img[138:629, 138:629]

        cv2.imwrite(output_name, img_cropped)
        #cv2.waitKey(0)
        #cv2.destroyWindow('test')

In [ ]:
# turn cube to gray labels

In [5]:
def np_discretize_to_order_labels(ar):
    bins = np.array([-0.125,0.125, 0.375, 0.625, 0.875,1.125])
    inds = np.digitize(ar, bins)
    ar_discret = (inds-1)/4
    
    return ar_discret

In [6]:
def Cube_to_Gray_Label(dir_in,dir_out):
    for pngfile in glob.glob(dir_in): #rendered PNG path
        
        #read images
        input_name = str(pngfile)[15:]
        example=cv2.imread(pngfile, cv2.IMREAD_COLOR)
        output_name = dir_out+input_name

        #k-means clustering
        img_data = example / 255.0
        img_data = img_data.reshape((-1, 3))

        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,100000, 0.00001)
        flags = cv2.KMEANS_RANDOM_CENTERS
        img_data = img_data.astype(np.float32)
        compactness, labels, centers = cv2.kmeans(img_data,5, None, criteria,10, flags)#number of clusters (classes: 5)

        centers[:,:] = np.round(centers[:,:])
        new_colors = centers[labels].reshape((-1, 3))

        example_recolored = new_colors.reshape(example.shape) #recolored sample

        #RGB map to grayscale map
        import pandas as pd
        w=example.shape[0]
        palette = np.array([[0, 0, 0],[0, 1, 1],[0, 0, 1],[0, 1, 0],[1, 1, 1]]) #order: black edge, ground, opaque building, glazing, sky
        matches = np.array([j for i in range(len(centers)) for j in range(len(palette)) if centers[i].tolist()==palette[j].tolist()]).reshape(-1,1)
        labels_ordered = matches[labels].reshape((w,w))
        labels_gray = np.array(labels_ordered*0.25)
        gray_img = cv2.cvtColor(labels_gray.astype(np.float32), cv2.COLOR_GRAY2BGR)#grayscale sample

        resized = cv2.resize(gray_img, (128,128), interpolation = cv2.INTER_AREA)#resze to 128*128
        
        re_dig = np_discretize_to_order_labels(resized)

        cv2.imwrite(output_name,re_dig*256)
        print(output_name)

# Start image processing

In [7]:
# start to turn original renderings to small size (128) grayscale images 
# Zurich
zh_list = glob.glob("zh/*.png")
To_Gray_Label_given_list(zh_list,"zh_gray/zh_")

zh_gray/zh_0.PNG
zh_gray/zh_1.PNG
zh_gray/zh_10.PNG
zh_gray/zh_100.PNG
zh_gray/zh_101.PNG
zh_gray/zh_102.PNG
zh_gray/zh_103.PNG
zh_gray/zh_104.PNG
zh_gray/zh_105.PNG
zh_gray/zh_106.PNG
zh_gray/zh_107.PNG
zh_gray/zh_108.PNG
zh_gray/zh_109.PNG
zh_gray/zh_11.PNG
zh_gray/zh_110.PNG
zh_gray/zh_111.PNG
zh_gray/zh_112.PNG
zh_gray/zh_113.PNG
zh_gray/zh_114.PNG
zh_gray/zh_115.PNG
zh_gray/zh_116.PNG
zh_gray/zh_117.PNG
zh_gray/zh_118.PNG
zh_gray/zh_119.PNG
zh_gray/zh_12.PNG
zh_gray/zh_120.PNG
zh_gray/zh_121.PNG
zh_gray/zh_122.PNG
zh_gray/zh_123.PNG
zh_gray/zh_124.PNG
zh_gray/zh_125.PNG
zh_gray/zh_126.PNG
zh_gray/zh_127.PNG
zh_gray/zh_128.PNG
zh_gray/zh_129.PNG
zh_gray/zh_13.PNG
zh_gray/zh_130.PNG
zh_gray/zh_131.PNG
zh_gray/zh_132.PNG
zh_gray/zh_133.PNG
zh_gray/zh_134.PNG
zh_gray/zh_135.PNG
zh_gray/zh_136.PNG
zh_gray/zh_137.PNG
zh_gray/zh_138.PNG
zh_gray/zh_139.PNG
zh_gray/zh_14.PNG
zh_gray/zh_140.PNG
zh_gray/zh_141.PNG
zh_gray/zh_142.PNG
zh_gray/zh_143.PNG
zh_gray/zh_144.PNG
zh_gray/zh_145.PNG
zh

In [ ]:
# start to turn original renderings to small size (128) grayscale images 
# Zurich
zh_list = glob.glob("zh/*.png")
To_Gray_Label_given_list(zh_list,"zh_gray/zh_")

In [7]:
# Singapore
sg_list = glob.glob("sg/*.png")
To_Gray_Label_given_list(sg_list,"sg_gray/sg_")

sg_gray/sg_0.PNG
sg_gray/sg_1.PNG
sg_gray/sg_10.PNG
sg_gray/sg_100.PNG
sg_gray/sg_101.PNG
sg_gray/sg_102.PNG
sg_gray/sg_103.PNG
sg_gray/sg_104.PNG
sg_gray/sg_105.PNG
sg_gray/sg_106.PNG
sg_gray/sg_107.PNG
sg_gray/sg_108.PNG
sg_gray/sg_109.PNG
sg_gray/sg_11.PNG
sg_gray/sg_110.PNG
sg_gray/sg_111.PNG
sg_gray/sg_112.PNG
sg_gray/sg_113.PNG
sg_gray/sg_114.PNG
sg_gray/sg_115.PNG
sg_gray/sg_116.PNG
sg_gray/sg_117.PNG
sg_gray/sg_118.PNG
sg_gray/sg_119.PNG
sg_gray/sg_12.PNG
sg_gray/sg_120.PNG
sg_gray/sg_121.PNG
sg_gray/sg_122.PNG
sg_gray/sg_123.PNG
sg_gray/sg_124.PNG
sg_gray/sg_125.PNG
sg_gray/sg_126.PNG
sg_gray/sg_127.PNG
sg_gray/sg_128.PNG
sg_gray/sg_129.PNG
sg_gray/sg_13.PNG
sg_gray/sg_130.PNG
sg_gray/sg_131.PNG
sg_gray/sg_132.PNG
sg_gray/sg_133.PNG
sg_gray/sg_134.PNG
sg_gray/sg_135.PNG
sg_gray/sg_136.PNG
sg_gray/sg_137.PNG
sg_gray/sg_138.PNG
sg_gray/sg_139.PNG
sg_gray/sg_14.PNG
sg_gray/sg_140.PNG
sg_gray/sg_141.PNG
sg_gray/sg_142.PNG
sg_gray/sg_143.PNG
sg_gray/sg_144.PNG
sg_gray/sg_145.PNG
sg

In [6]:
# start to turn original renderings to small size (128) grayscale images 
# Zurich-test
zh_list = glob.glob("zh/*.PNG")
#print(zh_list)
To_Gray_Label_given_list(zh_list,"zh_gray/zh_")

zh_gray/zh_12000.PNG
zh_gray/zh_12001.PNG
zh_gray/zh_12002.PNG
zh_gray/zh_12003.PNG
zh_gray/zh_12004.PNG
zh_gray/zh_12005.PNG
zh_gray/zh_12006.PNG
zh_gray/zh_12007.PNG
zh_gray/zh_12008.PNG
zh_gray/zh_12009.PNG
zh_gray/zh_12010.PNG
zh_gray/zh_12011.PNG
zh_gray/zh_12012.PNG
zh_gray/zh_12013.PNG
zh_gray/zh_12014.PNG
zh_gray/zh_12015.PNG
zh_gray/zh_12016.PNG
zh_gray/zh_12017.PNG
zh_gray/zh_12018.PNG
zh_gray/zh_12019.PNG
zh_gray/zh_12020.PNG
zh_gray/zh_12021.PNG
zh_gray/zh_12022.PNG
zh_gray/zh_12023.PNG
zh_gray/zh_12024.PNG
zh_gray/zh_12025.PNG
zh_gray/zh_12026.PNG
zh_gray/zh_12027.PNG
zh_gray/zh_12028.PNG
zh_gray/zh_12029.PNG
zh_gray/zh_12030.PNG
zh_gray/zh_12031.PNG
zh_gray/zh_12032.PNG
zh_gray/zh_12033.PNG
zh_gray/zh_12034.PNG
zh_gray/zh_12035.PNG
zh_gray/zh_12036.PNG
zh_gray/zh_12037.PNG
zh_gray/zh_12038.PNG
zh_gray/zh_12039.PNG
zh_gray/zh_12040.PNG
zh_gray/zh_12041.PNG
zh_gray/zh_12042.PNG
zh_gray/zh_12043.PNG
zh_gray/zh_12044.PNG
zh_gray/zh_12045.PNG
zh_gray/zh_12046.PNG
zh_gray/zh_12

In [11]:
# Singapore
sg_list = glob.glob("sg/*.png")
To_Gray_Label_given_list(sg_list,"sg_gray/sg_")

sg_gray/sg_12000.PNG
sg_gray/sg_12001.PNG
sg_gray/sg_12002.PNG
sg_gray/sg_12003.PNG
sg_gray/sg_12004.PNG
sg_gray/sg_12005.PNG
sg_gray/sg_12006.PNG
sg_gray/sg_12007.PNG
sg_gray/sg_12008.PNG
sg_gray/sg_12009.PNG
sg_gray/sg_12010.PNG
sg_gray/sg_12011.PNG
sg_gray/sg_12012.PNG
sg_gray/sg_12013.PNG
sg_gray/sg_12014.PNG
sg_gray/sg_12015.PNG
sg_gray/sg_12016.PNG
sg_gray/sg_12017.PNG
sg_gray/sg_12018.PNG
sg_gray/sg_12019.PNG
sg_gray/sg_12020.PNG
sg_gray/sg_12021.PNG
sg_gray/sg_12022.PNG
sg_gray/sg_12023.PNG
sg_gray/sg_12024.PNG
sg_gray/sg_12025.PNG
sg_gray/sg_12026.PNG
sg_gray/sg_12027.PNG
sg_gray/sg_12028.PNG
sg_gray/sg_12029.PNG
sg_gray/sg_12030.PNG
sg_gray/sg_12031.PNG
sg_gray/sg_12032.PNG
sg_gray/sg_12033.PNG
sg_gray/sg_12034.PNG
sg_gray/sg_12035.PNG
sg_gray/sg_12036.PNG
sg_gray/sg_12037.PNG
sg_gray/sg_12038.PNG
sg_gray/sg_12039.PNG
sg_gray/sg_12040.PNG
sg_gray/sg_12041.PNG
sg_gray/sg_12042.PNG
sg_gray/sg_12043.PNG
sg_gray/sg_12044.PNG
sg_gray/sg_12045.PNG
sg_gray/sg_12046.PNG
sg_gray/sg_12

In [ ]:
# start to turn fisheye renderings to cube

In [10]:
fisheye2dual(256,256,"zh_gray/*.png","zh/","zh_cube/zh_cube_")

In [8]:
fisheye2dual(256,256,"sg_gray/*.png","sg/","sg_cube/sg_cube_")

In [10]:
fisheye2dual(256,256,"zh_gray/*.png","zh/","zh_cube/zh_cube_")

In [13]:
fisheye2dual(256,256,"sg_gray/*.png","sg/","sg_cube/sg_cube_")

In [ ]:
# start to turn cube to gray labels

In [11]:
Cube_to_Gray_Label("zh_cube/*.png","zh_graycube/zh_graycube_")

zh_graycube/zh_graycube__0.PNG
zh_graycube/zh_graycube__1.PNG
zh_graycube/zh_graycube__10.PNG
zh_graycube/zh_graycube__100.PNG
zh_graycube/zh_graycube__101.PNG
zh_graycube/zh_graycube__102.PNG
zh_graycube/zh_graycube__103.PNG
zh_graycube/zh_graycube__104.PNG
zh_graycube/zh_graycube__105.PNG
zh_graycube/zh_graycube__106.PNG
zh_graycube/zh_graycube__107.PNG
zh_graycube/zh_graycube__108.PNG
zh_graycube/zh_graycube__109.PNG
zh_graycube/zh_graycube__11.PNG
zh_graycube/zh_graycube__110.PNG
zh_graycube/zh_graycube__111.PNG
zh_graycube/zh_graycube__112.PNG
zh_graycube/zh_graycube__113.PNG
zh_graycube/zh_graycube__114.PNG
zh_graycube/zh_graycube__115.PNG
zh_graycube/zh_graycube__116.PNG
zh_graycube/zh_graycube__117.PNG
zh_graycube/zh_graycube__118.PNG
zh_graycube/zh_graycube__119.PNG
zh_graycube/zh_graycube__12.PNG
zh_graycube/zh_graycube__120.PNG
zh_graycube/zh_graycube__121.PNG
zh_graycube/zh_graycube__122.PNG
zh_graycube/zh_graycube__123.PNG
zh_graycube/zh_graycube__124.PNG
zh_graycube/zh_gr

In [19]:
Cube_to_Gray_Label("zh_cube/*.png","zh_graycube/zh_graycube_")

zh_graycube/zh_graycube__12000.PNG
zh_graycube/zh_graycube__12001.PNG
zh_graycube/zh_graycube__12002.PNG
zh_graycube/zh_graycube__12003.PNG
zh_graycube/zh_graycube__12004.PNG
zh_graycube/zh_graycube__12005.PNG
zh_graycube/zh_graycube__12006.PNG
zh_graycube/zh_graycube__12007.PNG
zh_graycube/zh_graycube__12008.PNG
zh_graycube/zh_graycube__12009.PNG
zh_graycube/zh_graycube__12010.PNG
zh_graycube/zh_graycube__12011.PNG
zh_graycube/zh_graycube__12012.PNG
zh_graycube/zh_graycube__12013.PNG
zh_graycube/zh_graycube__12014.PNG
zh_graycube/zh_graycube__12015.PNG
zh_graycube/zh_graycube__12016.PNG
zh_graycube/zh_graycube__12017.PNG
zh_graycube/zh_graycube__12018.PNG
zh_graycube/zh_graycube__12019.PNG
zh_graycube/zh_graycube__12020.PNG
zh_graycube/zh_graycube__12021.PNG
zh_graycube/zh_graycube__12022.PNG
zh_graycube/zh_graycube__12023.PNG
zh_graycube/zh_graycube__12024.PNG
zh_graycube/zh_graycube__12025.PNG
zh_graycube/zh_graycube__12026.PNG
zh_graycube/zh_graycube__12027.PNG
zh_graycube/zh_grayc

In [9]:
Cube_to_Gray_Label("sg_cube/*.png","sg_graycube/sg_graycube_")

sg_graycube/sg_graycube__0.PNG
sg_graycube/sg_graycube__1.PNG
sg_graycube/sg_graycube__10.PNG
sg_graycube/sg_graycube__100.PNG
sg_graycube/sg_graycube__101.PNG
sg_graycube/sg_graycube__102.PNG
sg_graycube/sg_graycube__103.PNG
sg_graycube/sg_graycube__104.PNG
sg_graycube/sg_graycube__105.PNG
sg_graycube/sg_graycube__106.PNG
sg_graycube/sg_graycube__107.PNG
sg_graycube/sg_graycube__108.PNG
sg_graycube/sg_graycube__109.PNG
sg_graycube/sg_graycube__11.PNG
sg_graycube/sg_graycube__110.PNG
sg_graycube/sg_graycube__111.PNG
sg_graycube/sg_graycube__112.PNG
sg_graycube/sg_graycube__113.PNG
sg_graycube/sg_graycube__114.PNG
sg_graycube/sg_graycube__115.PNG
sg_graycube/sg_graycube__116.PNG
sg_graycube/sg_graycube__117.PNG
sg_graycube/sg_graycube__118.PNG
sg_graycube/sg_graycube__119.PNG
sg_graycube/sg_graycube__12.PNG
sg_graycube/sg_graycube__120.PNG
sg_graycube/sg_graycube__121.PNG
sg_graycube/sg_graycube__122.PNG
sg_graycube/sg_graycube__123.PNG
sg_graycube/sg_graycube__124.PNG
sg_graycube/sg_gr

In [20]:
Cube_to_Gray_Label("sg_cube/*.png","sg_graycube/sg_graycube_")

sg_graycube/sg_graycube__12000.PNG
sg_graycube/sg_graycube__12001.PNG
sg_graycube/sg_graycube__12002.PNG
sg_graycube/sg_graycube__12003.PNG
sg_graycube/sg_graycube__12004.PNG
sg_graycube/sg_graycube__12005.PNG
sg_graycube/sg_graycube__12006.PNG
sg_graycube/sg_graycube__12007.PNG
sg_graycube/sg_graycube__12008.PNG
sg_graycube/sg_graycube__12009.PNG
sg_graycube/sg_graycube__12010.PNG
sg_graycube/sg_graycube__12011.PNG
sg_graycube/sg_graycube__12012.PNG
sg_graycube/sg_graycube__12013.PNG
sg_graycube/sg_graycube__12014.PNG
sg_graycube/sg_graycube__12015.PNG
sg_graycube/sg_graycube__12016.PNG
sg_graycube/sg_graycube__12017.PNG
sg_graycube/sg_graycube__12018.PNG
sg_graycube/sg_graycube__12019.PNG
sg_graycube/sg_graycube__12020.PNG
sg_graycube/sg_graycube__12021.PNG
sg_graycube/sg_graycube__12022.PNG
sg_graycube/sg_graycube__12023.PNG
sg_graycube/sg_graycube__12024.PNG
sg_graycube/sg_graycube__12025.PNG
sg_graycube/sg_graycube__12026.PNG
sg_graycube/sg_graycube__12027.PNG
sg_graycube/sg_grayc

In [3]:
sorted(glob.glob('sg_graycube/*.png'))

['sg_graycube\\sg_graycube__12000.PNG',
 'sg_graycube\\sg_graycube__12001.PNG',
 'sg_graycube\\sg_graycube__12002.PNG',
 'sg_graycube\\sg_graycube__12003.PNG',
 'sg_graycube\\sg_graycube__12004.PNG',
 'sg_graycube\\sg_graycube__12005.PNG',
 'sg_graycube\\sg_graycube__12006.PNG',
 'sg_graycube\\sg_graycube__12007.PNG',
 'sg_graycube\\sg_graycube__12008.PNG',
 'sg_graycube\\sg_graycube__12009.PNG',
 'sg_graycube\\sg_graycube__12010.PNG',
 'sg_graycube\\sg_graycube__12011.PNG',
 'sg_graycube\\sg_graycube__12012.PNG',
 'sg_graycube\\sg_graycube__12013.PNG',
 'sg_graycube\\sg_graycube__12014.PNG',
 'sg_graycube\\sg_graycube__12015.PNG',
 'sg_graycube\\sg_graycube__12016.PNG',
 'sg_graycube\\sg_graycube__12017.PNG',
 'sg_graycube\\sg_graycube__12018.PNG',
 'sg_graycube\\sg_graycube__12019.PNG',
 'sg_graycube\\sg_graycube__12020.PNG',
 'sg_graycube\\sg_graycube__12021.PNG',
 'sg_graycube\\sg_graycube__12022.PNG',
 'sg_graycube\\sg_graycube__12023.PNG',
 'sg_graycube\\sg_graycube__12024.PNG',


In [6]:
np.random.rand(5)*2+0.3

array([1.5583493 , 0.4769277 , 1.31341994, 1.74497929, 1.75820585])

In [7]:
import numpy as np

mon_vector = [5,4,4,5,4,4,5,4,4,5,4,4]

past_day1 = np.sum(mon_vector[:5])*7
past_day1

mon_1_ix = [ix*364 + past_day1 for ix in range(1000)]

In [8]:
mon_1_ix

[154,
 518,
 882,
 1246,
 1610,
 1974,
 2338,
 2702,
 3066,
 3430,
 3794,
 4158,
 4522,
 4886,
 5250,
 5614,
 5978,
 6342,
 6706,
 7070,
 7434,
 7798,
 8162,
 8526,
 8890,
 9254,
 9618,
 9982,
 10346,
 10710,
 11074,
 11438,
 11802,
 12166,
 12530,
 12894,
 13258,
 13622,
 13986,
 14350,
 14714,
 15078,
 15442,
 15806,
 16170,
 16534,
 16898,
 17262,
 17626,
 17990,
 18354,
 18718,
 19082,
 19446,
 19810,
 20174,
 20538,
 20902,
 21266,
 21630,
 21994,
 22358,
 22722,
 23086,
 23450,
 23814,
 24178,
 24542,
 24906,
 25270,
 25634,
 25998,
 26362,
 26726,
 27090,
 27454,
 27818,
 28182,
 28546,
 28910,
 29274,
 29638,
 30002,
 30366,
 30730,
 31094,
 31458,
 31822,
 32186,
 32550,
 32914,
 33278,
 33642,
 34006,
 34370,
 34734,
 35098,
 35462,
 35826,
 36190,
 36554,
 36918,
 37282,
 37646,
 38010,
 38374,
 38738,
 39102,
 39466,
 39830,
 40194,
 40558,
 40922,
 41286,
 41650,
 42014,
 42378,
 42742,
 43106,
 43470,
 43834,
 44198,
 44562,
 44926,
 45290,
 45654,
 46018,
 46382,
 46746,

In [10]:
for i in range(4,5):
    print(i)

4
